# 第3章 字典和集合

## 3.1 泛映射类型
collections.abc模块中有Mapping和MutableMapping这两个抽象类, 它们的作用就是为dict和其它类似的类型定义形式接口。

In [5]:
import collections
my_dict = {}
isinstance(my_dict, collections.abc.Mapping)

True

标准库中所有的映射类型都是由dict实现的, 因此它们有个共同的限制就是, 只有可散列的数据类型才能作为Key。

In [6]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [8]:
tl = (1, 2, [30, 40])
hash(tl)  # 元组只有当其中所有的元素都是hashable的时候才能散列

TypeError: unhashable type: 'list'

## 3.2 字典推导
字典推导可以从任何以键值作为元素的可迭代对象中构建出字典。

In [9]:
DIAL_CODES = [
    (86, "China"),
    (91, "India"),
    (1, "United States")
]

In [10]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1}

## 3.3 常见的映射方法
书中的表格展示了dict, defaultdict和OrderedDict的常见方法。后面两个数据类型是dict的变种, 位于collections模块之内。
在本书之中介绍了setdefault()方法

## 3.4 映射的弹性查询
有时候方便起见, 就选在某个键在映射里面不存在, 我们也希望返回一个默认值。除了get方法, 我们从类的角度一般有两种实现方式: 一是使用defaultdict

这个类型, 二是自己定义一个dict子类, 然后在子类中实现\_\_missing\_\_方法。

### 3.4.1 defaultdict处理找不到键的一个选择

In [11]:
import collections

In [12]:
index = collections.defaultdict(list)  # 构造的时候传入一个default_factory函数(一个无参数的函数)

In [15]:
index["aa"].append(1)

In [16]:
index

defaultdict(list, {'aa': [1]})

In [17]:
c = collections.defaultdict()  # 开始没传入default_factory的话, 后续查询会报错

In [18]:
c["cc"]

KeyError: 'cc'

defaultdict的default_factory只会在\_\_getitem\_\_里被调用, 其实真正的原理是调用missing特殊函数,在其他方法中不会发挥作用。

### 3.4.2 特殊方法\_\_missing\_\_
所有映射类型在处理找不到的键时, 都会牵扯到\_\_missing\_\_。这个方法会在\_\_getitem\_\_找不到的时候发挥作用, 而不让程序抛出KeyError

In [58]:
# 下面自定义实现一个StrKeyDict0类, 将非字符串的key自动转换为字符串
class StrKeyDict0(dict):
    
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)  # 如果本来就是字符串的话, 找不到就直接抛出异常
        return self[str(key)]  # 如果是数字这类的话, 转为str在调用getitem
    
    
    def get(self, key, default = None):
        try:
            return self[key]  # 这里直接把get委托给getitem
        except KeyError:
            return defult
        
    
    def __contains__(self, key):
        return key in self or str(key) in self

In [29]:
str_key_dict = StrKeyDict0({"2": "two", "4": "four"})

In [30]:
str_key_dict[2]  # 查询数字

'two'

In [31]:
str_key_dict[1]  # 查询不存在的键

KeyError: '1'

In [32]:
print(str_key_dict.get(2))

two


## 3.5 字典的变种
在标准库collections中, 除了defaultdict之外还有一些变种类型：
* OrderedDict类型, 这个类型会在添加key的时候保持顺序
* ChainMap类型可以把多个映射对象连在一起
* Counter会给键准备一个整数计数器, 每次更新的时候都会增加这个计数器
* UserDict这个类就是把标准的dict类型用Python实现了一遍, 主要目的是让用户写子类去继承

In [36]:
import collections

In [39]:
# ChainMap
import builtins
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))
# pylookup

In [42]:
# Counter
ct = collections.Counter("abracadabra")
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [43]:
ct.update("aaaaa")
ct

Counter({'a': 10, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [44]:
ct.most_common(2)

[('a', 10), ('b', 2)]

## 3.6 子类化UserDict

In [59]:
# 在这里改进上面的StrKeyDict0类
class StrKeyDict(collections.UserDict):
    
    
    def __setitem__(self, key, item):
        # 因为dict实际上是c实现的, 所以我们如果继承dict的话, 就难以实现这个方法(会递归调用)
        self.data[str(key)] = item
        
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    
    def __contains__(self, key):
        return str(key) in self.data

## 3.7 不可变映射类型
可以使用MappingProxyType这个只读的代理类

In [60]:
from types import MappingProxyType
d = {"A": 1}
d_proxy = MappingProxyType(d)

In [61]:
d_proxy

mappingproxy({'A': 1})

In [62]:
d_proxy["A"] = 2  # 只读

TypeError: 'mappingproxy' object does not support item assignment

In [63]:
d["a"] = 1
d_proxy  # 同步更新

mappingproxy({'A': 1, 'a': 1})

## 3.8 集合论

In [64]:
l = ["spam", "spam", "eggs", "eggs"]
set(l)  # set中的类型必须是可散列的, 但是set本身不可散列(fronzenset可以)

{'eggs', 'spam'}

### 3.8.1 集合字面量

In [66]:
s = {1}
s

{1}

In [68]:
s = frozenset([1,2])
s

frozenset({1, 2})

### 3.8.2 集合推导

In [72]:
from unicodedata import name  # 查找字符的名称
{chr(i) for i in range(32, 256) if "SIGN" in name(chr(i), "")}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

从Python3.3开始, str,byte和datetime引入了随机加盐的过程。